In [ ]:
!pip uninstall -y pyarrow datasets
!pip install --no-use-pep517 pyarrow
!pip install datasets
!pip install torch transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install scikit-learn

In [23]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import s3fs
import os
from sklearn.model_selection import train_test_split
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

# Lister les fichiers d'un challenge
fs.ls("civel/diffusion/hackathon-minarm-2024/AIVSAI/hack_train.csv")

# Télécharger les données dans le service
PATH_IN = 'civel/diffusion/hackathon-minarm-2024/AIVSAI/hack_train.csv'
fs.download(PATH_IN, 'data/hack_train.csv')

df = pd.read_csv('data/hack_train.csv')
df

# Create a Hugging Face Dataset from the data (if not already in one)
data_dict = {
    "texts": df['text'],
    "labels": df['label']
}
hf_dataset = Dataset.from_dict(data_dict)

# Déjà préparé vos données dans hf_dataset
texts = hf_dataset["texts"]
labels = hf_dataset["labels"]
# Vectorisation des textes
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Split des données en ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, np.array(labels), test_size=0.2, random_state=42)


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8506687785990849


/opt/mamba/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
from sklearn.metrics import confusion_matrix, classification_report


# Obtenir la matrice de confusion et le rapport de classification
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Matrice de confusion:\n", conf_matrix)
print("\nRapport de classification:\n", class_report)

Matrice de confusion:
 [[4713  839]
 [ 858 4954]]

Rapport de classification:
               precision    recall  f1-score   support

           0       0.85      0.85      0.85      5552
           1       0.86      0.85      0.85      5812

    accuracy                           0.85     11364
   macro avg       0.85      0.85      0.85     11364
weighted avg       0.85      0.85      0.85     11364



In [29]:
# Assuming your DataFrame is named df
source_counts = df['src'].value_counts()

# Display the counts for each source
print("Source Counts:")
print(source_counts)

Source Counts:
src
hswag_human                                 3292
xsum_human                                  3283
roct_human                                  3275
eli5_human                                  3156
wp_human                                    3099
                                            ... 
hswag_machine_specified_text-davinci-003      57
squad_machine_continuation_gpt_neox           55
sci_gen_machine_continuation_gpt_j            54
yelp_machine_continuation_t0_11b              42
sci_gen_machine_continuation_gpt_neox         36
Name: count, Length: 322, dtype: int64


Prediction: [0]


In [30]:
df

,text,label,src
0,Little disclaimer: this deals with US laws and...,1,cmv_human
1,"Read: Mentally Retarded Downs. See, we've got ...",1,cmv_human
2,"If any of you frequent rbadhistory, there is a...",1,cmv_human
3,"I believe in a flat tax system, where everyone...",1,cmv_human
4,"Edit: Ok guy's, my views have been changed on ...",1,cmv_human
...,...,...,...
56814,We consider the recovery of a source term f (x...,1,sci_gen_human
56815,"Self-supervised learning (SlfSL), aiming at le...",1,sci_gen_human
56816,Recurrent neural networks (RNNs) have achieved...,1,sci_gen_human
56817,Deep reinforcement learning (DRL) is a booming...,1,sci_gen_human


In [81]:
dataset = load_dataset("Hello-SimpleAI/HC3", name="all")
human_answers = dataset['train']['human_answers']
chatgpt_answers = dataset['train']['chatgpt_answers']
human_answers = [string for sublist in dataset['train']['human_answers'] for string in sublist if sublist]
text_inputs = [string for sublist in dataset['train']['chatgpt_answers'] for string in sublist if sublist]


In [82]:
# Initialize a counter variable to keep track of the count
count_of_ones = 0

# Assuming you have a list of strings named text_inputs

for text_input in text_inputs:
    # Vectorize the text input using the same vectorizer
    text_input_vectorized = vectorizer.transform([text_input])

    # Make predictions using the trained model
    prediction = model.predict(text_input_vectorized)

    # Increment the counter if the prediction is 1
    if prediction == 0:
        count_of_ones += 1

# Print the count of sentences where the prediction is 1
print("Number of sentences with prediction 0:", count_of_ones)
print(count_of_ones/len(text_inputs)*100)


Number of sentences with prediction 0: 13944
51.830650856781766


36.89974726200506


In [70]:
print(len(text_inputs))

842
